# Testing out Javascript inside a notebook
> Notes while experimenting with running Javascript inside a Jupyter notebook. I took several approaches with varying degrees of success. This blog post is the publishing of the notebook with these experiments.

- toc:true
- branch: master
- badges: true
- author: Nissan Dookeran
- categories: [jupyter, javascript, experiments]
- show_tags: true
- comments: true

## First attempt - Using Pixiedust and Pixiedust_node

First we're going to borrow from [a notebook IBM published](https://github.com/IBM/nodejs-in-notebooks/blob/master/notebooks/nodebook_1.ipynb) on getting this approach up and running

In [1]:
# install or upgrade the packages
# restart the kernel to pick up the latest version
!pip install pixiedust --upgrade
!pip install pixiedust_node --upgrade

Requirement already up-to-date: pixiedust in c:\tools\anaconda3\envs\fastai2\lib\site-packages (1.1.18)
Requirement already up-to-date: pixiedust_node in c:\tools\anaconda3\envs\fastai2\lib\site-packages (0.2.5)


Now let's try activating what we just installed and running some javascript code thru Node. I already have Node installed on my local machine, but if you need it, [install Node from here](https://nodejs.org/en/download/)

In [2]:
import pixiedust_node

Pixiedust database opened successfully


In [3]:
%%node
print(new Date());

pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.
"2020-03-31T11:29:22.434Z"


Wow that's pretty cool. I will have to read up a little more into the T&C for this, since it seems to be tracking usage in my initial run of the import cell, but the message disappeared when I re-ran it, but I'm not too concerned on this yet since this is just experimentation.

So let's push a little and see what the limits of this integration can do. I was looking for a WYSIWYG Rich Text Editor to embed in an app, and I came across [Quill](https://github.com/quilljs/quill/). I don't want to invest too much time in getting an app running to see what it does, so let's see if we can embed the [Quill getting started sample code](https://quilljs.com/docs/quickstart/) here for a quick experiment with it. I keep forgetting to put `%%node` at the top of the code cell to run Javascript syntax so hopefully this note here fixes that.

In [11]:
%%node
// Remember to use javascript syntax from here forth
var htmlString= `
<!-- Include Quill stylesheet -->
<link href="https://cdn.quilljs.com/1.0.0/quill.snow.css" rel="stylesheet">

<!-- Create the toolbar container -->
<div id="toolbar">
  <button class="ql-bold">Bold</button>
  <button class="ql-italic">Italic</button>
</div>

<!-- Create the editor container -->
<div id="editor">
  <p>Hello World!</p>
</div>

<!-- Include the Quill library -->
<script src="https://cdn.quilljs.com/1.0.0/quill.js"></script>

<!-- Initialize Quill editor -->
<script type="text/javascript">
  var editor = new Quill('#editor', {
    modules: { toolbar: '#toolbar' },
    theme: 'snow'
  });
</script>
`
// console.log(htmlString)
html(htmlString)

... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...


Damn, maybe a bit too ambitious a start to think it could do this. Maybe it's the call to the external JS and Stylesheets. I'll have to investigate the why of this later. Let me just start with just the simple HTML part of that code and see if this (as uninteresting as it is) will run without error.

In [5]:
%%node
var str = `
<!-- Create the editor container -->
<div id="editor">  
  <p>Hello World!</p> 
  <p>Some initial <strong>bold</strong> text</p>
  <p><br></p>
</div>
`
html(str)

... ... ... ... ... ... ...


Nothing spectacular, but it is respecting HTML tags so that's something to note, let's move on. The documentation for `pixiedust_node` says I can see what npm packages are available and I can also install `npm` packages. Let's take a look what I have, and then try to install another WYSIWYG editor I noted called [Froala](https://github.com/froala/wysiwyg-editor-node-sdk) and see if we fare better here.

In [6]:
%%node
npm.list()


Uncaught


In [7]:
%%node
npm install froala-editor

npm should be run outside of the node repl, in your normal shell.
(Press Control-D to exit.)


Ok. So not getting the messages or ease of use I was expecting. Maybe it's my inexperience with the environment, but I was hoping for better. The pixiedust and pixiedust node combination have some merits, but aren't quite as glamorous as I hoped they'd be. Let's do some searching and try another approach. If I come back to doing more here and update the notebook I'll add a note cell here so the blog post reflects that anything from here is new stuff I added since first published

## Alternative approach: ReactJS code sample found in a Github gist

I found a [public gist on Github](https://gist.github.com/jeremyschlatter/c35c6bfa568e5a40440cb2fefcc7fd4e) that showed a notebook proporting to run ReactJS code. Let's give it a try, the code may be copied verbatim since I'm not sure yet what it's doing so don't want to break it apart yet, although I might do this in a future review of this.

In [15]:
import json
from string import Template

from IPython.display import Javascript, display
from IPython.core.magic import register_cell_magic


display(Javascript('''
// Load our libraries from a CDN instead of wherever this notebook is hosted.
require.config({
    paths: {
        babel: 'https://unpkg.com/babel-standalone@6/babel.min',
        react: 'https://unpkg.com/react@15.3.1/dist/react',
        'react-dom': 'https://unpkg.com/react-dom@15.3.1/dist/react-dom'
    }
})

// Hook to call into Python.
// Credit to disarticulate for documenting the usage of iopub: 
//    https://gist.github.com/disarticulate/d06069ff3e71cf828e5329beab8cb084
window.python = code => new Promise((resolve, reject) => {
    IPython.notebook.kernel.execute(
        code,
        {iopub: {output: data => resolve(data.content.text)}},
    )   
})
'''))


@register_cell_magic
def jsx(line, cell):
    display(Javascript((Template('''
        require(['babel', 'react', 'react-dom'], (Babel, React, ReactDOM) => {
            eval(Babel.transform($quoted_script, {presets: ['react']}).code)
        })
    ''').substitute(quoted_script=json.dumps(cell)))))

<IPython.core.display.Javascript object>

So the first time I ran the above cell I got no errors, but the 2nd time it generated a heap of them. I'm not sure if this is an after-effect of the failed previous experiment, but let's press on and hope the right bits got compiled as needed. The gist basically says I can set a python variable and then have the JSX declared later use this variable in what it displays. Let's give this a go using their sample

In [13]:
greeting = 'Hello from Python and React!'

In [14]:
%%jsx
python('print(greeting)').then(greeting => {
    ReactDOM.render(
      <h1>{greeting}</h1>,
      element[0],
    )
})

<IPython.core.display.Javascript object>

Ok it was pretty sweet that the first time I ran the code it rendered, but subsequent running spit out the gob of errors you see above. I wonder if it will work with a more complex bit of code though. I'll have to dig further and update this post in future with what I find here. There seem to be a few more approaches I can try, and even a few [Javascript kernels for Jupyter](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels) online to experiment with.

> Experiment status: **In progress**